In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [2]:
import numpy as np
import tensorflow as tf
import json

In [3]:
with open('dataset-bpe.json') as fopen:
    data = json.load(fopen)

In [4]:
train_X = data['train_X']
train_Y = data['train_Y']
test_X = data['test_X']
test_Y = data['test_Y']

In [5]:
EOS = 1
vocab_size = 32000

In [6]:
train_Y = [i + [EOS] for i in train_Y]
test_Y = [i + [EOS] for i in test_Y]
train_X = [i + [EOS] for i in train_X]
test_X = [i + [EOS] for i in test_X]

In [7]:
from collections import defaultdict

BASE_PARAMS = defaultdict(
    lambda: None,  # Set default value to None.

    # Input params
    default_batch_size=2048,  # Maximum number of tokens per batch of examples.
    default_batch_size_tpu=32768,
    max_length=256,  # Maximum number of tokens per example.

    # Model params
    initializer_gain=1.0,  # Used in trainable variable initialization.
    vocab_size=vocab_size,  # Number of tokens defined in the vocabulary file.
    hidden_size=512,  # Model dimension in the hidden layers.
    num_hidden_layers=6,  # Number of layers in the encoder and decoder stacks.
    num_heads=8,  # Number of heads to use in multi-headed attention.
    filter_size=2048,  # Inner layer dimension in the feedforward network.

    # Dropout values (only used when training)
    layer_postprocess_dropout=0.1,
    attention_dropout=0.1,
    relu_dropout=0.1,

    # Training params
    label_smoothing=0.1,
    learning_rate=1.0,
    learning_rate_decay_rate=1.0,
    learning_rate_warmup_steps=16000,

    # Optimizer params
    optimizer_adam_beta1=0.9,
    optimizer_adam_beta2=0.997,
    optimizer_adam_epsilon=1e-09,

    # Default prediction params
    extra_decode_length=50,
    beam_size=4,
    alpha=0.6,  # used to calculate length normalization in beam search

    # TPU specific parameters
    use_tpu=False,
    static_batch=False,
    allow_ffn_pad=True,
)

In [8]:
from tensor2tensor.utils import beam_search
from transformer.transformer import Transformer
from transformer import utils

def pad_second_dim(x, desired_size):
    padding = tf.tile([[[0.0]]], tf.stack([tf.shape(x)[0], desired_size - tf.shape(x)[1], tf.shape(x)[2]], 0))
    return tf.concat([x, padding], 1)

class Translator:
    def __init__(self, train = True, learning_rate = 1e-4):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype = tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype = tf.int32)
        batch_size = tf.shape(self.X)[0]
        
        model = Transformer(BASE_PARAMS, train)
        self.training_logits = model(self.X, self.Y)
        self.training_logits = self.training_logits[:, :tf.reduce_max(self.Y_seq_len)]
        self.training_logits = pad_second_dim(self.training_logits, tf.reduce_max(self.Y_seq_len))
        
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
#         self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
#                                                      targets = self.Y,
#                                                      weights = masks)
        
        xentropy, weights = utils.padded_cross_entropy_loss(
            self.training_logits, self.Y, BASE_PARAMS["label_smoothing"], BASE_PARAMS["vocab_size"])
        self.cost = tf.reduce_sum(xentropy) / tf.reduce_sum(weights)
        self.xentropy = xentropy
        self.weights = weights
        
#         optimizer = tf.contrib.opt.LazyAdamOptimizer(
#             BASE_PARAMS['learning_rate'],
#             beta1=BASE_PARAMS["optimizer_adam_beta1"],
#             beta2=BASE_PARAMS["optimizer_adam_beta2"],
#             epsilon=BASE_PARAMS["optimizer_adam_epsilon"])
        
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate,
                                               beta1=BASE_PARAMS["optimizer_adam_beta1"],
                                                beta2=BASE_PARAMS["optimizer_adam_beta2"],
                                                epsilon=BASE_PARAMS["optimizer_adam_epsilon"]).minimize(self.cost)
        
#         global_step = tf.train.get_global_step()
#         tvars = tf.trainable_variables()
#         gradients = optimizer.compute_gradients(
#             self.cost, tvars, colocate_gradients_with_ops=True)
#         minimize_op = optimizer.apply_gradients(
#             gradients, global_step=global_step, name="train")
#         update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
#         self.optimizer = tf.group(minimize_op, update_ops)

        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        self.fast_result = model(self.X)

In [9]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Translator(train = False)
sess.run(tf.global_variables_initializer())

Instructions for updating:
reduction_indices is deprecated, use axis instead


Instructions for updating:
Use `tf.cast` instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [10]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [11]:
batch_x = pad_sequences(train_X[:10], padding='post')
batch_y = pad_sequences(train_Y[:10], padding='post')

sess.run([model.fast_result, model.cost, model.accuracy], 
         feed_dict = {model.X: batch_x, model.Y: batch_y})

[{'outputs': array([[ 7843,  7843,  7843,  7843,  7843,  7843,  7843,  7843,  7843,
           7843,  7843,  7843,  7843,  7843,  7843,  7843,  7843,  7843,
           7843, 29821, 29821, 29821, 29821, 29821, 29821, 29821, 29821,
          29821, 29821, 29821, 29821, 29821, 29821, 29821, 29821, 29821,
          29821, 29821, 29821, 29821, 29821, 29821, 29821, 29821, 29821,
          29821, 29821, 29821, 29821, 29821, 29821, 29821, 29821, 29821,
          29821, 11702, 11702, 11702, 11702, 11702, 11702, 11702, 11702,
          11702, 11702, 11702, 11702, 11702, 11702, 11702, 11702, 11702,
          11702, 11702, 11702, 11702, 11702, 11702, 11702, 11702, 11702,
          11702, 11702, 11702, 11702, 11702, 11702],
         [ 7843,  7843,  7843,  7843,  7843,  7843,  7843,  7843,  7843,
           7843,  7843,  7843,  7843,  7843,  7843,  7843,  7843,  7843,
           7843,  7843,  7843,  7843,  7843,  7843,  7843,  7843,  7843,
           7843,  7843,  7843,  7843,  7843,  7843,  7843,  

In [12]:
batch_size = 128
epoch = 20

In [13]:
import tqdm

for e in range(epoch):
    pbar = tqdm.tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    train_loss, train_acc, test_loss, test_acc = [], [], [], []
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x = pad_sequences(train_X[i : index], padding='post')
        batch_y = pad_sequences(train_Y[i : index], padding='post')
        feed = {model.X: batch_x,
                model.Y: batch_y}
        accuracy, loss, _ = sess.run([model.accuracy,model.cost,model.optimizer],
                                    feed_dict = feed)
        train_loss.append(loss)
        train_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    
    pbar = tqdm.tqdm(
        range(0, len(test_X), batch_size), desc = 'minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x = pad_sequences(test_X[i : index], padding='post')
        batch_y = pad_sequences(test_Y[i : index], padding='post')
        feed = {model.X: batch_x,
                model.Y: batch_y,}
        accuracy, loss = sess.run([model.accuracy,model.cost],
                                    feed_dict = feed)

        test_loss.append(loss)
        test_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    print('epoch %d, training avg loss %f, training avg acc %f'%(e+1,
                                                                 np.mean(train_loss),np.mean(train_acc)))
    print('epoch %d, testing avg loss %f, testing avg acc %f'%(e+1,
                                                              np.mean(test_loss),np.mean(test_acc)))

minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 1, training avg loss 4.931033, training avg acc 0.231605
epoch 1, testing avg loss 3.965092, testing avg acc 0.325697


minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 2, training avg loss 3.495378, training avg acc 0.379644
epoch 2, testing avg loss 3.223244, testing avg acc 0.413091


minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

...

minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 17, training avg loss 0.621172, training avg acc 0.896736
epoch 17, testing avg loss 3.123955, testing avg acc 0.478847


minibatch loop:   0%|          | 0/1563 [00:00<?, ?it/s]

epoch 18, training avg loss 0.569504, training avg acc 0.911538
epoch 18, testing avg loss 3.145874, testing avg acc 0.483298


minibatch loop:  62%|██████▏   | 964/1563 [06:04<03:33,  2.81it/s, accuracy=0.925, cost=0.529]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
from tensor2tensor.utils import bleu_hook

In [18]:
results = []
for i in tqdm.tqdm(range(0, len(test_X), batch_size)):
    index = min(i + batch_size, len(test_X))
    batch_x = pad_sequences(test_X[i : index], padding='post')
    feed = {model.X: batch_x}
    p = sess.run(model.fast_result,feed_dict = feed)['outputs']
    result = []
    for row in p:
        result.append([i for i in row if i > 1])
    results.extend(result)

100%|██████████| 40/40 [03:12<00:00,  4.81s/it]


In [19]:
rights = []
for r in test_Y:
    rights.append([i for i in r if i > 1])

In [20]:
bleu_hook.compute_bleu(reference_corpus = rights,
                       translation_corpus = results)

0.17100729